<a href="https://colab.research.google.com/github/CryptoRobotFr/General-code-trading-bot/blob/main/All_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation et importation des données

In [ ]:
!pip install ta python-binance pandas_ta

In [12]:
import pandas as pd
from binance.client import Client
import ta
import numpy as np
import pandas_ta as pda
import requests

# Variables pour les données
client = Client()
pair_symbol = "BTCUSDT"
time_interval = Client.KLINE_INTERVAL_1DAY
start_date = "01 january 2017"

# Récupération des données
klinesT = client.get_historical_klines(pair_symbol, time_interval, start_date)

# Créer un tableau grâce aux données
df = pd.DataFrame(klinesT, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])

# Supprime les colonnes inutiles
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# Convertit les colonnes en numéric
for col in df.columns:
    df[col] = pd.to_numeric(df[col])

# Convertit les dates dans un format lisible
df = df.set_index(df['timestamp'])
df.index = pd.to_datetime(df.index, unit='ms')
del df['timestamp']

# Affiche le tableau
df

,open,high,low,close,volume
timestamp,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...
2022-04-30,38596.11,38795.38,37578.20,37630.80,35321.189890
2022-05-01,37630.80,38675.00,37386.38,38468.35,38812.241040
2022-05-02,38468.35,39167.34,38052.00,38525.16,53200.926280


# Les différents styles de bougie

In [13]:
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# Heiki Ashi Bar
def heikinashi_df(df):
    df['ha_close'] = (df.open + df.high + df.low + df.close)/4
    ha_open = [(df.open[0] + df.close[0]) / 2]
    [ha_open.append((ha_open[i] + df.ha_close.values[i]) / 2)
     for i in range(0, len(df)-1)]
    df['ha_open'] = ha_open
    df['ha_high'] = df[['ha_open', 'ha_close', 'high']].max(axis=1)
    df['ha_low'] = df[['ha_open', 'ha_close', 'low']].min(axis=1)
    return df

df = heikinashi_df(df)

#---------------------------- #
df

,open,high,low,close,volume,ha_close,ha_open,ha_high,ha_low
timestamp,,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,4308.1725,4273.280000,4485.390000,4200.74
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,4175.9350,4290.726250,4371.520000,3938.77
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,4070.7600,4233.330625,4233.330625,3850.00
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,4112.7425,4152.045313,4211.080000,4032.62
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,4029.1350,4132.393906,4132.393906,3911.79
...,...,...,...,...,...,...,...,...,...
2022-04-30,38596.11,38795.38,37578.20,37630.80,35321.189890,38150.1225,39214.923960,39214.923960,37578.20
2022-05-01,37630.80,38675.00,37386.38,38468.35,38812.241040,38040.1325,38682.523230,38682.523230,37386.38
2022-05-02,38468.35,39167.34,38052.00,38525.16,53200.926280,38553.2125,38361.327865,39167.340000,38052.00


#Les différents indicateurs de tendance

In [15]:
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# EMA
df['ema7']=ta.trend.ema_indicator(close=df['close'], window=7)
df['ema30']=ta.trend.ema_indicator(close=df['close'], window=30)
df['ema50']=ta.trend.ema_indicator(close=df['close'], window=50)
df['ema100']=ta.trend.ema_indicator(close=df['close'], window=100)
df['ema150']=ta.trend.ema_indicator(close=df['close'], window=150)
df['ema200']=ta.trend.ema_indicator(close=df['close'], window=200)
#------------------------- #

# SMA
df['sma7']=ta.trend.sma_indicator(close=df['close'], window=7)
#---------------------------- #



# Supertrend
# Classe de définition
class SuperTrend():
    def __init__(
        self,
        high,
        low,
        close,
        atr_window=10,
        atr_multi=3
    ):
        self.high = high
        self.low = low
        self.close = close
        self.atr_window = atr_window
        self.atr_multi = atr_multi
        self._run()
        
    def _run(self):
        # calculate ATR
        price_diffs = [self.high - self.low, 
                    self.high - self.close.shift(), 
                    self.close.shift() - self.low]
        true_range = pd.concat(price_diffs, axis=1)
        true_range = true_range.abs().max(axis=1)
        # default ATR calculation in supertrend indicator
        atr = true_range.ewm(alpha=1/self.atr_window,min_periods=self.atr_window).mean() 
        # atr = ta.volatility.average_true_range(high, low, close, atr_period)
        # df['atr'] = df['tr'].rolling(atr_period).mean()
        
        # HL2 is simply the average of high and low prices
        hl2 = (self.high + self.low) / 2
        # upperband and lowerband calculation
        # notice that final bands are set to be equal to the respective bands
        final_upperband = upperband = hl2 + (self.atr_multi * atr)
        final_lowerband = lowerband = hl2 - (self.atr_multi * atr)
        
        # initialize Supertrend column to True
        supertrend = [True] * len(self.close)
        
        for i in range(1, len(self.close)):
            curr, prev = i, i-1
            
            # if current close price crosses above upperband
            if self.close[curr] > final_upperband[prev]:
                supertrend[curr] = True
            # if current close price crosses below lowerband
            elif self.close[curr] < final_lowerband[prev]:
                supertrend[curr] = False
            # else, the trend continues
            else:
                supertrend[curr] = supertrend[prev]
                
                # adjustment to the final bands
                if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                    final_lowerband[curr] = final_lowerband[prev]
                if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                    final_upperband[curr] = final_upperband[prev]

            # to remove bands according to the trend direction
            if supertrend[curr] == True:
                final_upperband[curr] = np.nan
            else:
                final_lowerband[curr] = np.nan
                
        self.st = pd.DataFrame({
            'Supertrend': supertrend,
            'Final Lowerband': final_lowerband,
            'Final Upperband': final_upperband
        })
        
    def super_trend_upper(self):
        return self.st['Final Upperband']
        
    def super_trend_lower(self):
        return self.st['Final Lowerband']
        
    def super_trend_direction(self):
        return self.st['Supertrend']

# Récupération des valeurs
st_atr_window = 20
st_atr_multiplier = 3

super_trend = SuperTrend(df['high'], df['low'], df['close'], st_atr_window, st_atr_multiplier)
df['super_trend_direction'] = super_trend.super_trend_direction() # True if bullish, False if bearish
df['super_trend_upper'] = super_trend.super_trend_upper()
df['super_trend_lower'] = super_trend.super_trend_lower()
#---------------------------- #


# TRIX
#Classe de féinition
class Trix():
    """ Trix indicator

        Args:
            close(pd.Series): dataframe 'close' columns,
            trixLength(int): the window length for each mooving average of the trix,
            trixSignal(int): the window length for the signal line
    """

    def __init__(
        self,
        close: pd.Series,
        trixLength: int = 9,
        trixSignal: int = 21
    ):
        self.close = close
        self.trixLength = trixLength
        self.trixSignal = trixSignal
        self._run()

    def _run(self):
        self.trixLine = ta.trend.ema_indicator(
            ta.trend.ema_indicator(
                ta.trend.ema_indicator(
                    close=self.close, window=self.trixLength),
                window=self.trixLength), window=self.trixLength)
        self.trixPctLine = self.trixLine.pct_change()*100
        self.trixSignalLine = ta.trend.sma_indicator(
            close=self.trixPctLine, window=self.trixSignal)
        self.trixHisto = self.trixPctLine - self.trixSignalLine

    def trix_line(self) -> pd.Series:
        """ trix line

            Returns:
                pd.Series: trix line
        """
        return pd.Series(self.trixLine, name="TRIX_LINE")

    def trix_pct_line(self) -> pd.Series:
        """ trix percentage line

            Returns:
                pd.Series: trix percentage line
        """
        return pd.Series(self.trixPctLine, name="TRIX_PCT_LINE")

    def trix_signal_line(self) -> pd.Series:
        """ trix signal line

            Returns:
                pd.Series: trix siganl line
        """
        return pd.Series(self.trixSignal, name="TRIX_SIGNAL_LINE")

    def trix_histo(self) -> pd.Series:
        """ trix histogram

            Returns:
                pd.Series: trix histogram
        """
        return pd.Series(self.trixHisto, name="TRIX_HISTO")

# Récupération des valeurs
trix_length = 9
trix_signal = 21
trix = Trix(df["close"], trix_length, trix_signal)
df['trix_pct_line'] = trix.trix_pct_line()
df['trix_signal_line'] = trix.trix_signal_line()
df['trix_histo'] = trix.trix_histo()

#------------------------ #


# Ichimoku
df['kijun'] = ta.trend.ichimoku_base_line(df['high'], df['low'])
df['tenkan'] = ta.trend.ichimoku_conversion_line(df['high'], df['low'])
df['ssa'] = ta.trend.ichimoku_a(df['high'], df['low'])
df['ssb'] = ta.trend.ichimoku_b(df['high'], df['low'])
df['ssa25'] = ta.trend.ichimoku_a(df['high'], df['low']).shift(25)
df['ssb25'] = ta.trend.ichimoku_b(df['high'], df['low']).shift(25)
df['ssa52'] = ta.trend.ichimoku_a(df['high'], df['low']).shift(50)
df['ssb52'] = ta.trend.ichimoku_b(df['high'], df['low']).shift(50)
df['close25'] = df['close'].shift(25)
df['close1'] = df['close'].shift(1)
#----------------------- #


# MACD
macd = ta.trend.MACD(close=df['close'], window_fast=12, window_slow=26, window_sign=9)
df['macd'] = macd.macd()
df['macd_signal'] = macd.macd_signal()
df['macd_histo'] = macd.macd_diff() #Histogramme MACD
#----------------------- #


# Awesome Oscillator
df['awesome_oscilllator'] = ta.momentum.awesome_oscillator(high=df['high'], low=df['low'], window1=5, window2=34)
#---------------------- #


# ADX
df['adx'] =ta.trend.adx(high=df['high'], low=df['low'], close = df['close'], window = 14)
#----------------------- #

# Fear and Greed 
# Défintion
def fear_and_greed(close):
    ''' Fear and greed indicator
    '''
    response = requests.get("https://api.alternative.me/fng/?limit=0&format=json")
    dataResponse = response.json()['data']
    fear = pd.DataFrame(dataResponse, columns = ['timestamp', 'value'])

    fear = fear.set_index(fear['timestamp'])
    fear.index = pd.to_datetime(fear.index, unit='s')
    del fear['timestamp']
    df = pd.DataFrame(close, columns = ['close'])
    df['fearResult'] = fear['value']
    df['FEAR'] = df['fearResult'].ffill()
    df['FEAR'] = df.FEAR.astype(float)
    return pd.Series(df['FEAR'], name="FEAR")

# Récupération des valeurs
df["f_g"] = fear_and_greed(df["close"])

df

/usr/local/lib/python3.7/dist-packages/ta/trend.py:769: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.7/dist-packages/ta/trend.py:774: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


,open,high,low,close,volume,ema7,ema30,ema50,ema100,ema150,...,ssa52,ssb52,close25,close1,macd,macd_signal,macd_histo,awesome_oscilllator,adx,f_g
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4285.08,NaN,NaN,NaN,NaN,0.000000,NaN
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4108.37,NaN,NaN,NaN,NaN,0.000000,NaN
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4139.98,NaN,NaN,NaN,NaN,0.000000,NaN
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,4086.29,NaN,NaN,NaN,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30,38596.11,38795.38,37578.20,37630.80,35321.189890,38917.493567,40656.232153,41193.623537,42283.526262,43295.777133,...,40244.600,39369.085,45497.55,38596.11,-1046.505514,-892.013610,-154.491904,-3235.196000,26.629618,20.0
2022-05-01,37630.80,38675.00,37386.38,38468.35,38812.241040,38805.207675,40515.078466,41086.750065,42207.978217,43231.837701,...,39867.835,39369.085,43170.47,37630.80,-1045.290932,-922.669074,-122.621858,-3202.661765,27.482466,22.0
2022-05-02,38468.35,39167.34,38052.00,38525.16,53200.926280,38735.195756,40386.696629,40986.295553,42135.051124,43169.497599,...,39867.835,39369.085,43444.19,38468.35,-1027.895338,-943.714327,-84.181011,-2954.154588,27.651542,28.0


#Les différents oscillateurs 

In [21]:
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

# RSI
df['rsi'] = ta.momentum.RSIIndicator(close=df['close'], window=14)
#------------------------- #


# STOCHASTIC RSI
df['stoch_rsi'] = ta.momentum.stochrsi(close=df['close'], window=14)
df['stochastic'] = ta.momentum.stoch(high=df['high'],low=df['low'],close=df['close'], window=14,smooth_window=3)
df['stoch_signal'] =ta.momentum.stoch_signal(high =df['high'],low=df['low'],close=df['close'], window=14, smooth_window=3)
#------------------------- #


# VMC
#Class pour nos indicateurs
class VMC():
    """ VuManChu Cipher B + Divergences 
        Args:
            high(pandas.Series): dataset 'High' column.
            low(pandas.Series): dataset 'Low' column.
            close(pandas.Series): dataset 'Close' column.
            wtChannelLen(int): n period.
            wtAverageLen(int): n period.
            wtMALen(int): n period.
            rsiMFIperiod(int): n period.
            rsiMFIMultiplier(int): n period.
            rsiMFIPosY(int): n period.
    """
    def __init__(
        self: pd.Series,
        high: pd.Series,
        low: pd.Series,
        close: pd.Series,
        open: pd.Series,
        wtChannelLen: int = 9,
        wtAverageLen: int = 12,
        wtMALen: int = 3,
        rsiMFIperiod: int = 60,
        rsiMFIMultiplier: int = 150,
        rsiMFIPosY: int = 2.5
    ) -> None:
        self._high = high
        self._low = low
        self._close = close
        self._open = open
        self._wtChannelLen = wtChannelLen
        self._wtAverageLen = wtAverageLen
        self._wtMALen = wtMALen
        self._rsiMFIperiod = rsiMFIperiod
        self._rsiMFIMultiplier = rsiMFIMultiplier
        self._rsiMFIPosY = rsiMFIPosY
        self._run()
        self.wave_1()

    def _run(self) -> None:
        try:
            self._esa = ta.trend.ema_indicator(
                close=self._close, window=self._wtChannelLen)
        except Exception as e:
            print(e)
            raise

        self._esa = ta.trend.ema_indicator(
            close=self._close, window=self._wtChannelLen)
        self._de = ta.trend.ema_indicator(
            close=abs(self._close - self._esa), window=self._wtChannelLen)
        self._rsi = ta.trend.sma_indicator(self._close, self._rsiMFIperiod)
        self._ci = (self._close - self._esa) / (0.015 * self._de)

    def wave_1(self) -> pd.Series:
        """VMC Wave 1 
        Returns:
            pandas.Series: New feature generated.
        """
        wt1 = ta.trend.ema_indicator(self._ci, self._wtAverageLen)
        return pd.Series(wt1, name="wt1")

    def wave_2(self) -> pd.Series:
        """VMC Wave 2
        Returns:
            pandas.Series: New feature generated.
        """
        wt2 = ta.trend.sma_indicator(self.wave_1(), self._wtMALen)
        return pd.Series(wt2, name="wt2")

    def money_flow(self) -> pd.Series:
        """VMC Money Flow
            Returns:
            pandas.Series: New feature generated.
        """
        mfi = ((self._close - self._open) /
                (self._high - self._low)) * self._rsiMFIMultiplier
        rsi = ta.trend.sma_indicator(mfi, self._rsiMFIperiod)
        money_flow = rsi - self._rsiMFIPosY
        return pd.Series(money_flow, name="money_flow")

# Récupération des données
df['hlc3'] = (df['high'] +df['close'] + df['low'])/3
vmc = VMC(high =df['high'],low = df['low'],close=df['hlc3'],open=df['open'])
df['vmc_wave1'] = vmc.wave_1()
df['vmc_wave2'] = vmc.wave_2()
vmc = VMC(high=df['high'], low=df['low'], close=df['close'], open=df['open'])
df['money_flow'] = vmc.money_flow()
#------------------------- #


# WilliamsR
df['max_21'] = df['high'].rolling(21).max()
df['min_21'] = df['low'].rolling(21).min()
df['william_r'] = (df['close'] - df['max_21']) / (df['max_21'] - df['min_21']) * 100
df['emaw'] = ta.trend.ema_indicator(close=df['william_r'], window=13)
#--------------------------- #


# CCI
df['hlc3'] = (df['high'] + df['low'] + df['close']) / 3 
df['sma_cci'] = df['hlc3'].rolling(40).mean()
df['mad'] = df['hlc3'].rolling(40).apply(lambda x: pd.Series(x).mad())
df['cci'] = (df['hlc3'] - df['sma_cci']) / (0.015 * df['mad']) 
#--------------------------- #


# PPO
df['ppo'] = ta.momentum.ppo(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
df['ppo_signal'] = ta.momentum.ppo_signal(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
df['ppo_histo'] = ta.momentum.ppo_hist(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
#----------------------------- #


# PVO
df['pvo'] = ta.momentum.pvo(volume = df['volume'], window_slow=26, window_fast=12, window_sign=9)
df['pvo_signal'] = ta.momentum.pvo_signal(volume = df['volume'], window_slow=26, window_fast=12, window_sign=9)
df['pvo_histo'] = ta.momentum.pvo_hist(volume = df['volume'], window_slow=26, window_fast=12, window_sign=9)
#---------------------------- #


# Aroon
df['aroon_up'] = ta.trend.aroon_up(close=df['close'], window=25)
df['aroon_dow'] = ta.trend.aroon_down(close=df['close'], window=25)
#------------------------------#

df

,open,high,low,close,volume,rsi,stoch_rsi,stochastic,stoch_signal,hlc3,...,mad,cci,ppo,ppo_signal,ppo_histo,pvo,pvo_signal,pvo_histo,aroon_up,aroon_dow
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,NaN,NaN,NaN,4323.736667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,NaN,NaN,NaN,4139.553333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,NaN,NaN,NaN,4058.223333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,NaN,NaN,NaN,4109.996667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,NaN,NaN,NaN,4015.803333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30,38596.11,38795.38,37578.20,37630.80,35321.189890,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,0.077412,0.974471,18.867389,38001.460000,...,2456.247883,-119.402109,-2.585791,-2.176981,-0.408810,4.605919,2.760792,1.845127,8.0,100.0
2022-05-01,37630.80,38675.00,37386.38,38468.35,38812.241040,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,0.467431,19.356772,12.426772,38176.576667,...,2551.809254,-107.727751,-2.592294,-2.260044,-0.332250,2.912661,2.791166,0.121496,4.0,96.0
2022-05-02,38468.35,39167.34,38052.00,38525.16,53200.926280,<ta.momentum.RSIIndicator object at 0x7f58b8fc...,0.493856,20.373120,13.568121,38581.500000,...,2628.511433,-91.795665,-2.557600,-2.319555,-0.238045,4.062625,3.045457,1.017168,8.0,92.0


#Les différents indicateurs de volatilité/volume

In [22]:
df.drop(columns = df.columns.difference(['timestamp','open','high','low','close','volume']), inplace=True)

#Court code pour définir les différents indicateurs de volatilité à implementer dans vos stratégies de trading

# Kaufman ou KAMA
df["kama"] = ta.momentum.kama(df['close'], window=10, pow1=2, pow2=30)
# ------------------------- #


# ATR
df["atr"] = ta.volatility.average_true_range(df['high'], df['low'], df['close'], window=2)
# ------------------------- #


# Les bandes de Bollingers
df["bol_high"] = ta.volatility.bollinger_hband(df['close'], window=20, window_dev=2)
df["bol_low"] = ta.volatility.bollinger_lband(df['close'], window=20, window_dev=2)
df["bol_medium"] = ta.volatility.bollinger_mavg(df['close'], window=20)
df["bol_gap"] = ta.volatility.bollinger_wband(df['close'], window=20, window_dev=2)
# Return binaire 0 ou 1 
df["bol_higher"] = ta.volatility.bollinger_hband_indicator(df['close'], window=20, window_dev=2)
df["bol_lower"] = ta.volatility.bollinger_lband_indicator(df['close'], window=20, window_dev=2)
# ------------------------- #


# Le canal de Donchian
df["don_high"] = ta.volatility.donchian_channel_hband(df['high'], df['low'], df['close'], window=20, offset=0)
df["don_low"] = ta.volatility.donchian_channel_lband(df['high'], df['low'], df['close'], window=20, offset=0)
df["don_medium"] = ta.volatility.donchian_channel_mband(df['high'], df['low'], df['close'], window=20, offset=0)
# ------------------------- #


# Le canal de Keltner 
df["kel_high"] = ta.volatility.keltner_channel_hband(df['high'], df['low'], df['close'], window=20, window_atr=10)
df["kel_low"] = ta.volatility.keltner_channel_lband(df['high'], df['low'], df['close'], window=20, window_atr=10)
df["kel_medium"] = ta.volatility.keltner_channel_mband(df['high'], df['low'], df['close'], window=20 ,window_atr=10)
# Return binaire 0 ou 1 
df["kel_higher"] = ta.volatility.keltner_channel_hband_indicator(df['high'], df['low'], df['close'], window=20, window_atr=10)
df["kel_lower"] = ta.volatility.keltner_channel_lband_indicator(df['high'], df['low'], df['close'], window=20, window_atr=10)
# ------------------------- #


# Chopiness
def chop(high, low, close, window=14):
    ''' Choppiness indicator
    '''
    tr1 = pd.DataFrame(high - low).rename(columns={0: 'tr1'})
    tr2 = pd.DataFrame(abs(high - close.shift(1))
                       ).rename(columns={0: 'tr2'})
    tr3 = pd.DataFrame(abs(low - close.shift(1))
                       ).rename(columns={0: 'tr3'})
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis=1, join='inner').dropna().max(axis=1)
    atr = tr.rolling(1).mean()
    highh = high.rolling(window).max()
    lowl = low.rolling(window).min()
    chop_serie = 100 * np.log10((atr.rolling(window).sum()) /
                          (highh - lowl)) / np.log10(window)
    return pd.Series(chop_serie, name="CHOP")

df["chop"] = chop(df['high'], df['low'], df['close'], window=14)
# ------------------------- #


# ADI Indicator
df["adi"] = ta.volume.acc_dist_index(high=df['high'], low=df['low'], close=df['close'], volume = df['volume'])
#--------------------------- #


# FI
df["fi"] = ta.volume.force_index(close=df['close'] ,volume=df['volume'], window=13)

#Volume anomaly
def volume_anomalu(df, volume_window=10):
    dfInd = df.copy()
    dfInd["VolAnomaly"] = 0
    dfInd["PreviousClose"] = dfInd["close"].shift(1)
    dfInd['MeanVolume'] = dfInd['volume'].rolling(volume_window).mean()
    dfInd['MaxVolume'] = dfInd['volume'].rolling(volume_window).max()
    dfInd.loc[dfInd['volume'] > 1.5 * dfInd['MeanVolume'], "VolAnomaly"] = 1
    dfInd.loc[dfInd['volume'] > 2 * dfInd['MeanVolume'], "VolAnomaly"] = 2
    dfInd.loc[dfInd['volume'] >= dfInd['MaxVolume'], "VolAnomaly"] = 3
    dfInd.loc[dfInd['PreviousClose'] > dfInd['close'],
              "VolAnomaly"] = (-1) * dfInd["VolAnomaly"]
    return dfInd["VolAnomaly"]

df["volume_anomaly"] = volume_anomalu(df, volume_window=10)

df

,open,high,low,close,volume,kama,atr,bol_high,bol_low,bol_medium,...,don_medium,kel_high,kel_low,kel_medium,kel_higher,kel_lower,chop,adi,fi,volume_anomaly
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,NaN,0.000000,NaN,NaN,NaN,...,NaN,4608.386667,4039.086667,NaN,0.0,0.0,NaN,-3.239543e+02,NaN,0
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,NaN,358.700000,NaN,NaN,NaN,...,NaN,4590.345000,3872.945000,NaN,0.0,0.0,NaN,-5.833409e+02,NaN,0
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,NaN,346.695000,NaN,NaN,NaN,...,NaN,4524.534444,3823.141111,NaN,0.0,0.0,NaN,-3.039066e+02,NaN,0
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,NaN,262.577500,NaN,NaN,NaN,...,NaN,4465.515000,3850.240000,NaN,0.0,0.0,NaN,-4.900489e+02,NaN,0
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,NaN,235.203750,NaN,NaN,NaN,...,NaN,4417.138667,3841.786667,NaN,0.0,0.0,NaN,-4.880851e+02,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-30,38596.11,38795.38,37578.20,37630.80,35321.189890,40601.472505,1484.939984,41862.674028,37915.439972,39889.0570,...,40277.10,41689.999000,38218.907000,39954.453000,0.0,1.0,57.022841,8.749038e+06,-1.480242e+07,0
2022-05-01,37630.80,38675.00,37386.38,38468.35,38812.241040,40515.808147,1386.779992,41900.375766,37771.528234,39835.9520,...,40181.19,41483.437333,38204.954333,39844.195833,0.0,0.0,56.070557,8.775402e+06,-8.043907e+06,0
2022-05-02,38468.35,39167.34,38052.00,38525.16,53200.926280,40470.144506,1251.059996,41896.228045,37620.697955,39758.4630,...,40181.19,41395.584667,38150.004667,39772.794667,0.0,0.0,53.739126,8.767340e+06,-6.463014e+06,0
